In [1]:
from pymongo import MongoClient
import sqlite3
import pandas as pd
import joblib
from IPython.display import display, Markdown

In [2]:
rnaseq_srxs = joblib.load("../output/library_strategy-wf/rnaseq_inliers.pkl")
print(len(rnaseq_srxs))

13905


In [3]:
# Mongo DataBase
client = MongoClient()
db = client["sramongo"]["ncbi"]

# SQLLite DataBase
con = sqlite3.connect("../data/biometa.db")
cur = con.cursor()

In [4]:
display(Markdown("## Number of BioProjects"))
next(db.aggregate([
    {
        "$match": {
            "srx": {"$in": rnaseq_srxs}
        }
    },
    {
        "$group": {
            "_id": "$BioProject.accn",
            "title": {"$first": "$BioProject.title"}
        }
    },
    {"$count": "cnt"}
]))["cnt"]

## Number of BioProjects

595

In [5]:
display(Markdown("## Number of BioSamples"))
next(db.aggregate([
    {
        "$match": {
            "srx": {"$in": rnaseq_srxs}
        }
    },
    {
        "$group": {
            "_id": "$BioSample.accn"
        }
    },
    {"$count": "cnt"}
]))["cnt"]

## Number of BioSamples

12933

In [6]:
display(Markdown("## Number of BioSamples Complete"))
cur.execute("SELECT * FROM biometa WHERE complete = 1")
len(cur.fetchall())

## Number of BioSamples Complete

908

In [7]:
# Close database connections
client.close()
con.close()